In [3]:
# default_exp algo.seq.embeding.word2vec

%reload_ext autoreload
%autoreload 2

In [ ]:
algo-seq-embeding-word2vec

# 评估词向量
https://blog.csdn.net/TimEcho/article/details/106281209

* 评估方式一：降维。训练好的词向量，首先要进行降维，使用工具TSNE降维到2维，这样映射到二维空间，便于观察词向量训练后是否能将同类词聚集到一起
* 评估方式二：相似度对比，选定两个词向量，计算相似度，然后与人工标注的相似度进行对比
* 评估方式三：类比。计算woman和man的距离，然后在词库中找与girl距离跟woman和man距离相近的词

* 打印loss: 选loss最低不再增长
* 终极评估方式: 对于一些没有语义的向量，如item2vec，在训练的每n个epoch打印其最相近的n个items，如果基本稳定，不怎么变化，则说明训练好了

# word embeding


## word2vec中skipgram的缺点

1. 不考虑上下文，即同一个word只有一种向量表示，没有考虑一词多义的情况  
解决方法：context-aware word embedding:ELmo或者BERT
2. 窗口长度有限，  
解决：类似于LM，使用RNN，LSTM来学习出词向量
3. 无法考虑全局，  
解决方法：全局模型（MF矩阵分解）
4. 无法有效学习低频词向量，
5. 未登录词，只在训练集出现已经训练好词向量，但是测试集没有，（OOV：out-of-vacabulary）  
4,5，使用subword embedding解决
6. 严格意义上的语序，解决：LM（RNN/LSTM）  
7. 可解释性不足,解决：非欧式空间

## subword: 
解决低频词、OOV, 根据已有词向量学习出新词的向量

这种subword的方式是这样的：加入窗口大小n=4，将每个单词分成多个部分，每个部分包含4个character，  
训练是我已然是采样skipgram模型，所以得到的词向量是subword这种形式。那么对于未登录词，完全可以通过找词库中的这种subword词向量来拼接到一个未登录词。并得到他的条件概率。  
不过这种方法不适用于中文。  
论文：enriching word embedding via wubword information

当数据量很少时，可以考虑使用这种subword

# word2vec
https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

## bag-of-words model
This model transforms each document to a fixed-length vector of integers. For example, given the sentences:

    John likes to watch movies. Mary likes movies too.

    John also likes to watch football games. Mary hates football.

The model outputs the vectors:

    [1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]

    [1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]

Bag-of-words models are surprisingly effective, but have several weaknesses.

首先，他们会丢失所有有关单词顺序的信息：“约翰喜欢玛丽”和“玛丽喜欢约翰”对应于相同的向量。 There is a solution: bag of n-grams models consider word phrases of length n to represent documents as fixed-length vectors to capture local word order but suffer from data sparsity and high dimensionality.

其次，该模型不会尝试学习基础单词的含义，因此，向量之间的距离并不总是反映出含义上的差异。 

Word2Vec模型解决了第二个问题。

## the Word2Vec Model
Word2Vec是一种更新的模型，它使用浅层神经网络将单词嵌入到低维向量空间中。 结果是一组词向量，其中在向量空间中靠在一起的向量根据上下文具有相似的含义，而彼此远离的词向量具有不同的含义。

The are two versions of this model and Word2Vec class implements them both:

### Skip-grams (SG)
The Word2Vec Skip-gram model, for example, takes in pairs (word1, word2) generated by moving a window across text data, and trains a 1-hidden-layer neural network based on the synthetic task of given an input word, giving us a predicted probability distribution of nearby words to the input. A virtual one-hot encoding of words goes through a ‘projection layer’ to the hidden layer; these projection weights are later interpreted as the word embeddings. So if the hidden layer has 300 neurons, this network will give us 300-dimensional word embeddings.

### Continuous-bag-of-words (CBOW)
Continuous-bag-of-words Word2vec is very similar to the skip-gram model. It is also a 1-hidden-layer neural network. The synthetic training task now uses the average of multiple input context words, rather than a single word as in skip-gram, to predict the center word. Again, the projection weights that turn one-hot words into averageable vectors, of the same width as the hidden layer, are interpreted as the word embeddings.

# word2vec代码实现
## keras实现by 苏剑林
https://kexue.fm/archives/4515

# gensim.models.Word2Vec

In [23]:
# !pip install -U gensim
!pip freeze | grep gensim

gensim==3.7.2


## 默认参数

In [7]:
from gensim.models import Word2Vec
import gensim

In [ ]:
Word2Vec()

    Word2Vec(
        sentences=None,
        corpus_file=None,
        size=100,
        alpha=0.025,
        window=5,
        min_count=5,
        max_vocab_size=None,
        sample=0.001,
        seed=1,
        workers=3,
        min_alpha=0.0001,
        sg=0,
        hs=0,
        negative=5,
        ns_exponent=0.75,
        cbow_mean=1,
        hashfxn=<built-in function hash>,
        iter=5,
        null_word=0,
        trim_rule=None,
        sorted_vocab=1,
        batch_words=10000,
        compute_loss=False,
        callbacks=(),
        max_final_vocab=None,
    )
    
* iter : int, optional 默认是5，要调
    Number of iterations (epochs) over the corpus.
* alpha : float, optional
    The initial learning rate.
* min_alpha : float, optional
    Learning rate will linearly drop to `min_alpha` as training progresses.
* workers : int, optional  默认是3，所以训练时要手动设置一下
    Use these many worker threads to train the model (=faster training with multicore machines).
* min_count
min_count用于修剪内部字典。 在十亿个单词的语料库中仅出现一次或两次的单词可能是无趣的错别字和垃圾。 此外，没有足够的数据来对这些词进行任何有意义的训练，因此最好忽略它们：

min_count = 5的默认值
* size
size是gensim Word2Vec将单词映射到的N维空间的维数（N）。

较大的值需要更多的训练数据，但可以产生更好（更准确）的模型。 合理的值在数十到数百之间。

default value of size=100

* window=5 : int, optional
    Maximum distance between the current and predicted word within a sentence.

### callbacks
https://www.lizenghai.com/archives/16296.html

https://radimrehurek.com/gensim/models/callbacks.html

In [19]:
import time

import gensim

class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    """
    用于保存模型, 打印损失函数等等
    """
    def __init__(self, savedir='./', save_name='word2vector.model'):

        self.save_path = savedir+save_name

        self.epoch = 0

        self.pre_loss = 0

        self.best_loss = 999999999.9

        self.since = time.time()
    def on_epoch_begin(self, model):
        pass
    def on_epoch_end(self, model):
        
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        if self.epoch % 1 == 0:

            epoch_loss = cum_loss - self.pre_loss

            time_taken = time.time() - self.since

            print('Epoch %d, loss: %.2f, time: %dmin %ds' %

                        (self.epoch, epoch_loss, time_taken//60, time_taken%60))

            if self.best_loss > epoch_loss:

                self.best_loss = epoch_loss

                print('Better model.Best loss: %.2f' % self.best_loss)

                model.save(self.save_path)

                print('Model %s save done!' % self.save_path)

        self.pre_loss = cum_loss

        self.since = time.time()

## Training Your Own Model
这个语料库足够小，可以完全放入内存，但是我们将实现一个内存友好的迭代器，逐行读取它，以演示如何处理更大的语料库。

如果我们想进行任何自定义的预处理，例如 解码非标准编码，小写字母，删除数字，提取命名实体……所有这些都可以在MyCorpus迭代器中完成，而word2vec不需要知道。 所需要的就是输入产生一个句子（另一个utf8单词列表）。

In [5]:
from gensim import utils
from gensim.test.utils import datapath
from gensim.models import Word2Vec

In [49]:

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
#             print(line[:5])
            yield utils.simple_preprocess(line)

In [40]:
utils.simple_preprocess('Hundreds of people have been forced to vacate their homes')

['hundreds',
 'of',
 'people',
 'have',
 'been',
 'forced',
 'to',
 'vacate',
 'their',
 'homes']

In [61]:
sentences = MyCorpus()
model = Word2Vec(sentences=sentences, callbacks=[EpochSaver()], iter=50, compute_loss=True)

Epoch 5, loss: 100409.81, time: 0min 0s
Better model.Best loss: 100409.81
Model ./word2vector.model save done!


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch 10, loss: 94070.44, time: 0min 0s
Better model.Best loss: 94070.44
Model ./word2vector.model save done!


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch 15, loss: 86205.00, time: 0min 0s
Better model.Best loss: 86205.00
Model ./word2vector.model save done!


...

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch 45, loss: 62111.25, time: 0min 0s
Better model.Best loss: 62111.25
Model ./word2vector.model save done!


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch 50, loss: 61513.75, time: 0min 0s
Better model.Best loss: 61513.75
Model ./word2vector.model save done!


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Update the model's neural weights from a sequence of sentences.

In [33]:
model.train(sentences=sentences, total_examples=model.corpus_count, callbacks=[callback()], epochs=50)

Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 1: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 2: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 3: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 4: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 5: 0.0
for word king: the top 3 sim user is [('george', 0.46930772066116333), ('however', 0.41419118642807007), ('former', 0.41314512491226196)]
Loss after epoch 6: 0.0
for 

(1797065, 2907600)

每个训练周期内 回调没有变化, 手动更新。。。，总之回调函数没什么用

In [36]:
for i in range(10):
    model.train(sentences=sentences, total_examples=model.corpus_count, callbacks=[callback()], epochs=1)

Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('george', 0.3772778809070587), ('source', 0.374118447303772), ('former', 0.36147093772888184)]
Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('source', 0.36969128251075745), ('george', 0.3638962507247925), ('former', 0.3539872169494629)]
Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('source', 0.37612271308898926), ('george', 0.3636791706085205), ('former', 0.3475203812122345)]
Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('source', 0.37409070134162903), ('george', 0.36320599913597107), ('terror', 0.3548111021518707)]
Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('source', 0.36942291259765625), ('george', 0.3611094355583191), ('terror', 0.3507077693939209)]
Loss after epoch 0: 0.0
for word king: the top 3 sim user is [('source', 0.3696172833442688), ('george', 0.3632398843765259), ('terror', 0.3527523875236511)]
Loss after epoch 0: 0.0
for word king: the top 

In [66]:
# RuntimeError: you must first build vocabulary before training the model
# 原因:训练文本太小 min_count默认值为5，如果语料太小，导致每个词的发生频率都小于5，则会导致vocabulary为空。。。
sentences = [
    ["cat", "say", "meow"], 
    ["dog", "say", "woof"]]
model = gensim.models.Word2Vec(sentences=sentences, min_count=1)

### 获取词向量

In [13]:
vec_king = model.wv['king']
vec_king

array([-0.03686245, -0.01609651, -0.01444744, -0.02135829, -0.00200862,
       -0.01151453, -0.01819683,  0.02734437, -0.10053819, -0.05595972,
        0.04234225,  0.04281876,  0.02587122,  0.01270954,  0.04601529,
       -0.00835075,  0.00156381, -0.01310062,  0.00457986,  0.02177754,
        0.02796555,  0.03658348, -0.00421377,  0.02298167,  0.00075962,
       -0.01811693, -0.08794788, -0.01969494, -0.01617615, -0.01301005,
        0.03622155, -0.02132978,  0.00818219, -0.01280667, -0.06247395,
       -0.02782101,  0.01052102,  0.03244915,  0.055819  ,  0.02236952,
       -0.02228366, -0.06680909, -0.03983621, -0.02890607,  0.03852952,
       -0.05652092, -0.00395071, -0.065943  ,  0.0195525 ,  0.02234451,
        0.02618746, -0.01053842, -0.01325279, -0.03100039, -0.03426435,
        0.02069941, -0.04345337,  0.02748772, -0.02539084, -0.01945122,
       -0.01035313,  0.05203196,  0.01739551, -0.02849312, -0.04863365,
       -0.01096844, -0.06444025, -0.01036584, -0.04000191,  0.04

In [38]:
model['king']  # removed

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.01486553,  0.01823401, -0.02772865,  0.00213301,  0.0372683 ,
        0.01642814,  0.04816883, -0.05448446,  0.01253152,  0.03404462,
        0.01756212, -0.04840738, -0.03839137, -0.01304129, -0.03804687,
        0.05305994, -0.00860955, -0.00836853,  0.00134603, -0.03344008,
       -0.03039226, -0.03281286, -0.00623441, -0.02607584,  0.00408613,
        0.04711006,  0.03532708, -0.0055987 , -0.00130932,  0.02229508,
       -0.04697058, -0.02922403,  0.0131788 , -0.01622746,  0.05389683,
       -0.02905632,  0.03431457,  0.01364054,  0.00634684, -0.01930137,
       -0.01475715, -0.04367147, -0.00581169, -0.03634231,  0.01892559,
       -0.01157352,  0.02849645,  0.00408916,  0.03510711, -0.0240911 ,
       -0.03448397, -0.00420031, -0.06505472,  0.00562562, -0.03190516,
       -0.01666111, -0.04189085,  0.07402997,  0.00904749, -0.03249473,
        0.00972437,  0.03047183, -0.00820827, -0.02223486, -0.01516857,
       -0.00403475, -0.02458509,  0.02097399, -0.00187658,  0.01

#### OOV

In [34]:
try:
    vec_cameroon = model.wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


### 获取最相近的词向量topk

In [15]:
r = model.wv.most_similar('king', topn=2)
r

[('highway', 0.9974420070648193), ('reported', 0.9973129630088806)]

In [20]:
r = model.wv.most_similar('king1', topn=2)
r

KeyError: "word 'king1' not in vocabulary"

In [16]:
dict(r)

{'highway': 0.9974420070648193, 'reported': 0.9973129630088806}

In [17]:
import pandas as pd

/Users/luoyonggui/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/luoyonggui/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [18]:
pd.Series(r)

0     (highway, 0.9974420070648193)
1    (reported, 0.9973129630088806)
dtype: object

In [19]:
pd.Series(dict(r))

highway     0.997442
reported    0.997313
dtype: float64

In [36]:
# Print the 5 most similar words to “car” or “minivan”
print(model.wv.most_similar(positive=['car', 'man'], topn=5))

[('have', 0.9996969699859619), ('five', 0.9996781349182129), ('israeli', 0.9996719360351562), ('just', 0.9996703863143921), ('other', 0.999667763710022)]


In [37]:
model.wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car'])

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'land'

### 获取词汇表

In [67]:
model.wv.vocab

{'cat': <gensim.models.keyedvectors.Vocab at 0x12039d278>,
 'say': <gensim.models.keyedvectors.Vocab at 0x120392198>,
 'meow': <gensim.models.keyedvectors.Vocab at 0x120392cf8>,
 'dog': <gensim.models.keyedvectors.Vocab at 0x120392320>,
 'woof': <gensim.models.keyedvectors.Vocab at 0x120392048>}

In [69]:
model.vector_size

100

In [68]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

cat
say
meow
dog
woof


### Storing and loading models

#### Save the model.
This saved model can be loaded again using :func:`~gensim.models.word2vec.Word2Vec.load`, which supports
online training and getting vectors for vocabulary words.

如果model不大，则只会生成aa.model一个文件

而如果model很大，则会生成aa.model、aa.model.trainables.syn1neg.npy和aa.model.wv.vectors.npy三个文件

In [75]:
model.save('aa.model')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [32]:
new_model = Word2Vec.load('aa.model')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#### Save KeyedVectors
使用mdoel.wv.save以KededVectors实例的形式保存词向量文件，以该方式保存的模型丢失了完整的模型状态，无法再训练，保存的对象更小更快。

In [76]:
model.wv.save('aa.wv')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [81]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load("aa.wv", mmap='r')
# wv['cat'] # numpy vector of a word

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#### save_word2vec_format
Store the input-hidden weight matrix in the same format used by the original
C word2vec-tool, for compatibility.

In [72]:
model.wv.save_word2vec_format('aa.txt')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
aa.txt

第一行 词汇数 词向量维度

    5 100
    say -0.0010033615 -0.0009983357 -0.0046261796 ...
    cat 0.0037430483 -0.0044084494 -0.00355869 ...

In [73]:
# 文件大小会小一些
model.wv.save_word2vec_format('aa.bin', binary=True)

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [82]:
from gensim.models import KeyedVectors
wv_from_text = KeyedVectors.load_word2vec_format("aa.txt", binary=False) # C text format
wv_from_bin = KeyedVectors.load_word2vec_format("aa.bin", binary=True) # C bin format


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [99]:
wv_from_bin.index2word

['say', 'cat', 'meow', 'dog', 'woof']

In [98]:
wv_from_bin.vocab

{'say': <gensim.models.keyedvectors.Vocab at 0x11f6e04e0>,
 'cat': <gensim.models.keyedvectors.Vocab at 0x120299f28>,
 'meow': <gensim.models.keyedvectors.Vocab at 0x120299a58>,
 'dog': <gensim.models.keyedvectors.Vocab at 0x120299eb8>,
 'woof': <gensim.models.keyedvectors.Vocab at 0x120299e80>}

In [97]:
wv_from_bin.vector_size

100

In [96]:
wv_from_bin.vectors

array([[-1.00336154e-03, -9.98335658e-04, -4.62617958e-03,
        -3.38307256e-03, -3.09544732e-03,  1.51311210e-03,
         3.80300032e-03,  3.18969460e-03,  7.81343377e-04,
        -2.46190885e-03,  2.17640679e-03, -3.25980550e-03,
         1.43450543e-05,  4.70067002e-03,  1.95618439e-03,
         4.28239303e-03,  3.11228703e-03,  1.51620491e-03,
        -1.50276057e-03, -4.60313837e-04,  2.48095999e-03,
        -3.22831911e-03, -3.85779800e-04,  3.56054096e-03,
        -3.30647174e-03, -1.74928561e-03,  8.98306957e-04,
        -3.42513435e-03,  3.91588220e-03, -1.15593395e-03,
         3.85119743e-03, -3.03606829e-03,  1.79845863e-03,
        -4.82477993e-03,  3.07267765e-03, -2.03650724e-03,
         3.78798088e-03,  1.83291547e-03, -2.65652593e-03,
         1.30606335e-04,  1.59174844e-03,  2.87242723e-03,
        -4.25131060e-03,  3.60778859e-03, -4.82053263e-03,
        -5.82585286e-04, -3.77888908e-03,  6.24802196e-05,
         2.50778510e-04,  1.17574842e-03,  2.87227868e-0

In [95]:
wv_from_bin['cat']

array([ 3.7430483e-03, -4.4084494e-03, -3.5586900e-03, -2.1001576e-03,
        1.2616201e-03, -2.8212117e-03, -1.9504626e-03,  7.4059790e-04,
        3.1178794e-03, -4.4649048e-03, -4.0089744e-03,  6.8061007e-04,
        3.3125374e-03,  3.4610869e-03,  4.1681770e-03,  2.4661524e-03,
       -3.1603505e-03, -2.6616068e-03, -3.6751078e-03, -1.8300968e-03,
       -2.9618305e-03, -3.9897794e-03,  2.9524898e-03, -9.1363516e-05,
        2.6891907e-04,  3.9253986e-04,  4.6698353e-04, -4.7475174e-03,
        1.6226899e-03, -3.7550658e-03, -7.7973295e-04,  4.3094847e-03,
        2.9831608e-03,  3.9672633e-03,  1.6150601e-03, -4.8489897e-03,
       -4.9042054e-03, -2.2035427e-03,  1.0598595e-03, -4.4332191e-04,
        3.5333852e-04, -3.7603425e-03,  1.9409549e-03,  4.3727783e-03,
        3.1951910e-03,  3.1808000e-03,  4.0026279e-03, -1.1207482e-03,
        4.7568078e-03, -1.0089022e-03, -1.3066172e-03,  7.4233953e-04,
        2.6448215e-03, -3.5879829e-03, -4.1432315e-03, -2.2584722e-04,
      

In [89]:
e = wv_from_bin.get_keras_embedding()

In [93]:
e.weights

[]

In [92]:
e.get_weights()

[]

In [84]:
# wv_from_text['cat']
# wv_from_bin['cat']

array([ 3.7430483e-03, -4.4084494e-03, -3.5586900e-03, -2.1001576e-03,
        1.2616201e-03, -2.8212117e-03, -1.9504626e-03,  7.4059790e-04,
        3.1178794e-03, -4.4649048e-03, -4.0089744e-03,  6.8061007e-04,
        3.3125374e-03,  3.4610869e-03,  4.1681770e-03,  2.4661524e-03,
       -3.1603505e-03, -2.6616068e-03, -3.6751078e-03, -1.8300968e-03,
       -2.9618305e-03, -3.9897794e-03,  2.9524898e-03, -9.1363516e-05,
        2.6891907e-04,  3.9253986e-04,  4.6698353e-04, -4.7475174e-03,
        1.6226899e-03, -3.7550658e-03, -7.7973295e-04,  4.3094847e-03,
        2.9831608e-03,  3.9672633e-03,  1.6150601e-03, -4.8489897e-03,
       -4.9042054e-03, -2.2035427e-03,  1.0598595e-03, -4.4332191e-04,
        3.5333852e-04, -3.7603425e-03,  1.9409549e-03,  4.3727783e-03,
        3.1951910e-03,  3.1808000e-03,  4.0026279e-03, -1.1207482e-03,
        4.7568078e-03, -1.0089022e-03, -1.3066172e-03,  7.4233953e-04,
        2.6448215e-03, -3.5879829e-03, -4.1432315e-03, -2.2584722e-04,
      

# fasttext
https://mp.weixin.qq.com/s/MmikAgOSz7k9ncGkWIlMbA

## fastText 在不同语境中至少有两个含义
1. 在文章 Bag of Tricks for Efficient Text Classification [1] 中，fastText 是作者提出的文本分类器的名字。与 sub-word 无关！也不是新的词嵌入训练模型！是 word2vec 中 CBOW 模型的简单变种。
2. 作为 Facebook 开源包，fastText [6] 是用来训练词嵌入或句嵌入的，其不仅包括 1 中论文的代码实现，还包括 Enriching Word Vectors with Subword Information [2] 及 FastText.zip: Compressing text classification models [3] 两文的代码实现。

## 文本分类语义下的fasttext

### Bag of Tricks for Efficient Text Classification

In [1]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1607.01759.pdf', width=1000, height=450)

### FastText.zip: Compressing text classification models

In [3]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1612.03651.pdf', width=1000, height=450)

### Character-level Convolutional Networks for Text Classification

In [4]:
from IPython.display import IFrame
IFrame('https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf', width=1000, height=450)

## embedding语义下的fasttext

### Enriching Word Vectors with Subword Information

In [2]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1607.04606.pdf', width=1000, height=450)

## gensim-fasttext
https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py

In [6]:
from gensim.models.fasttext import FastText

In [ ]:
FastText(
    sentences=None,
    corpus_file=None,
    sg=0,
    hs=0,
    size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    word_ngrams=1,
    # {1,0}, optional
    # If 1, uses enriches word vectors with subword(n-grams) information.
    # If 0, this is equivalent to :class:`~gensim.models.word2vec.Word2Vec`.
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    iter=5,  # Number of epochs， 一般显然是不够的
    null_word=0,
    min_n=3,  # min length of char ngrams
    max_n=6,
    sorted_vocab=1,
    bucket=2000000,  # number of buckets used for hashing ngrams 
    trim_rule=None,
    batch_words=10000,
    callbacks=(),
    compatible_hash=True,
)

### training 中文
训练速度比word2vec要慢很多

In [48]:
from gensim.models import FastText
sentences = [["你好", "世界", "快乐"], ["好人", "世代", "快速"]]

model = FastText(sentences, min_count=1, min_n=2, iter=10)


In [49]:
model.wv['世界']  # get vector for word

array([ 2.9822402e-03,  2.4169697e-03,  1.7187260e-03, -2.2653495e-03,
        3.2119157e-03,  4.2951079e-03, -2.7774605e-03,  2.5285247e-03,
        7.1119861e-04, -2.2702373e-03, -2.2216819e-03, -1.1484689e-03,
       -5.0927966e-04,  2.7783182e-03, -3.7633646e-03,  2.2833713e-03,
        2.8870531e-04,  2.9109684e-03, -9.9741039e-04,  9.3552069e-04,
        6.5907091e-04, -1.2434857e-03, -4.8777787e-05, -5.5110501e-03,
       -8.8064518e-04, -1.8348387e-03, -3.1964940e-03,  1.9632792e-03,
        2.2461899e-03, -2.3505124e-03, -5.6509962e-03,  2.8363969e-03,
       -8.7185681e-04,  9.0928376e-04,  3.1893721e-03, -2.6449324e-03,
        4.7927522e-04, -2.9875131e-03,  3.3288699e-04, -2.3074101e-03,
       -1.2953918e-03, -8.3305809e-04,  4.7961317e-04, -2.4373434e-03,
       -2.1793812e-03,  8.0079201e-04, -3.0157524e-03,  4.0204329e-03,
        3.0183580e-04,  1.8538638e-03, -2.0454575e-03, -2.6678795e-03,
        1.1443186e-03, -3.0163454e-03,  7.0191745e-04,  5.2529224e-03,
      

In [50]:
model.wv['快快乐乐']  # get vector for out-of-vocab word

array([ 3.0318054e-04,  2.6340729e-03, -2.4031913e-03,  5.1210332e-04,
       -6.4614485e-04,  1.4889885e-03,  1.5180273e-03, -9.9588011e-04,
       -1.4411961e-03,  8.9846511e-04,  3.5744504e-04,  1.5467522e-03,
        3.7019928e-03, -1.4805131e-03, -2.1489430e-05, -1.8503825e-03,
        1.4395656e-03, -5.2286941e-04, -2.9034005e-03,  1.6823788e-03,
       -2.2158846e-03, -9.8364311e-04, -9.2589605e-04, -1.2630870e-03,
        5.3450739e-04, -8.9164951e-04,  2.0512906e-03, -3.8826707e-04,
        1.5873264e-03, -2.3737226e-03, -1.0957335e-03,  1.7603323e-03,
        3.6992301e-04,  2.5232488e-03,  8.5212628e-04, -1.5399997e-03,
       -1.7697698e-03,  1.4537339e-03,  5.7253614e-04, -3.4134286e-05,
       -1.7359739e-03,  1.1944456e-03, -4.4834069e-03,  2.2543869e-03,
       -7.1508944e-04,  9.7015040e-04,  4.5475870e-04,  9.7663095e-04,
        2.4216895e-03,  1.8361442e-03,  4.1975899e-04,  1.5667530e-03,
       -1.7469459e-03, -1.3007789e-03,  2.0048935e-03,  2.3161517e-03,
      

In [51]:
model.wv.similarity("快乐", "快快乐乐")

0.43810502

In [52]:
model.wv.similarity("世界", "快快乐乐")

-0.00832216

### Training models

In [9]:
from pprint import pprint as print
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

In [10]:
corpus_file

'/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/gensim/test/test_data/lee_background.cor'

In [29]:
model = FT_gensim(size=100)

In [30]:
# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [23]:
model.epochs

5

In [24]:
model.corpus_count

300

In [25]:
model.corpus_total_words

59890

In [31]:
# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words
)

In [32]:
print(model)

In [33]:
# Tests if word present in vocab
'night' in model.wv.vocab

True

In [34]:
'nights' in model.wv.vocab

False

In [36]:
model.wv['night']

array([ 8.71200413e-02,  1.05707003e-02, -5.89070261e-01,  5.15045047e-01,
        5.87517440e-01, -3.32078636e-01, -1.90413103e-01, -3.94908749e-02,
        4.24223274e-01,  3.16444933e-01, -6.34695113e-01, -2.14536432e-02,
       -6.66504323e-01,  4.09089714e-01,  3.09547037e-01, -5.73672540e-02,
       -1.86798245e-01,  2.07086176e-01,  2.25862935e-01, -3.73620391e-01,
       -2.31027037e-01,  3.02217811e-01, -3.93239588e-01,  1.71807073e-02,
       -8.23755801e-01,  7.31919646e-01,  1.30469099e-01,  1.77326202e-01,
        4.04274732e-01,  8.38539843e-03, -6.41942382e-01,  2.52658665e-01,
        8.58276263e-02, -5.00359297e-01,  4.58392501e-01,  1.12920761e-01,
       -1.48069695e-01, -7.77938142e-02,  4.22724515e-01,  2.39092767e-01,
       -5.21457894e-03, -7.26774260e-02,  3.92118275e-01, -7.20630363e-02,
        1.19034253e-01,  1.91644192e-01, -1.39423281e-01,  2.18378246e-01,
       -3.35901529e-02, -3.93209457e-01, -5.56832254e-01, -5.41895211e-01,
        5.90641387e-02, -

In [37]:
model.wv['nights']

array([ 0.07672799,  0.00973572, -0.51288426,  0.4476877 ,  0.51063144,
       -0.29056957, -0.16643377, -0.03358696,  0.36901104,  0.27614596,
       -0.5546457 , -0.02012125, -0.58049345,  0.35754701,  0.2655268 ,
       -0.04965462, -0.16471185,  0.17855662,  0.19501755, -0.32570162,
       -0.20110163,  0.26402733, -0.34305644,  0.01563679, -0.7180861 ,
        0.63717043,  0.11359484,  0.15489486,  0.35225046,  0.00689856,
       -0.5569709 ,  0.221257  ,  0.07347611, -0.4380085 ,  0.39914072,
        0.0980963 , -0.12754871, -0.06904901,  0.36826727,  0.20910455,
       -0.00362907, -0.06345739,  0.34115458, -0.06107789,  0.10262474,
        0.16722696, -0.12506145,  0.19204657, -0.02991424, -0.34114182,
       -0.48763385, -0.47133043,  0.05262994,  0.00075857,  0.352133  ,
       -0.6848708 , -0.09930194, -0.15151477,  0.01734567, -0.02622395,
        0.17119884, -0.09591019, -0.44669098, -0.07795443, -0.4646708 ,
        0.32806468,  0.01163389,  0.14541139,  0.01733191,  0.43

In [39]:
model.wv.similarity("night", "nights")

0.9999928

In [41]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

'breakfast'

In [43]:
model.wv.most_similar(positive=['baghdad', 'england'], negative=['london'])

[('1', 0.23961403965950012),
 ('40', 0.23430491983890533),
 ('2', 0.23235931992530823),
 ('26', 0.23015664517879486),
 ('20', 0.23013760149478912),
 ('UN', 0.2299460470676422),
 ('blaze', 0.22948595881462097),
 ('keep', 0.22878581285476685),
 ('As', 0.2280689775943756),
 ('...', 0.22806063294410706)]